In [5]:
!pip install scikit-optimize

In [9]:
# Testign GridSearch Cv and BayesianSearch Cv for hyperparameter tuning
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
import joblib
from skopt import BayesSearchCV  # This requires scikit-optimize
from skopt.space import Real, Integer, Categorical
import joblib 


In [ ]:
# =============================================================================
# STATISTICAL MODEL COMPARISON (HYPOTHESIS TESTING + EFFECT SIZE)
# =============================================================================

from scipy.stats import ttest_rel, wilcoxon, friedmanchisquare
import itertools
import numpy as np

def statistical_comparison(results_dict, alpha=0.05):
    """
    Perform pairwise statistical comparisons between models using:
    - Paired t-test (assumes normality of performance scores)
    - Wilcoxon signed-rank test (non-parametric, no distribution assumption)

    Parameters:
    -----------
    results_dict : dict
        Output from hyperparameter tuning. Each value must include 'cv_scores', e.g.:
        {
            "ModelA": {'cv_scores': [...]},
            "ModelB": {'cv_scores': [...]}
        }
    alpha : float (default=0.05)
        Significance level for testing (probability of Type I error)

    Returns:
    --------
    dict
        {
            "ModelA_vs_ModelB": {
                "mean_diff": value,
                "t_test": {"stat": , "p_value": },
                "wilcoxon": {"stat": , "p_value": },
                "significant_t": True/False,
                "significant_w": True/False
            }, ...
        }
    """

    print("STATISTICAL HYPOTHESIS TESTING")
    print("=" * 50)

    model_names = list(results_dict.keys())
    test_results = {}

    for model1, model2 in itertools.combinations(model_names, 2):
        scores1 = results_dict[model1]['cv_scores']
        scores2 = results_dict[model2]['cv_scores']

        # Paired t-test (parametric)
        t_stat, t_pval = ttest_rel(scores1, scores2)

        # Wilcoxon signed-rank test (non-parametric)
        w_stat, w_pval = wilcoxon(scores1, scores2)

        mean_diff = np.mean(scores1) - np.mean(scores2)

        test_results[f"{model1}_vs_{model2}"] = {
            'mean_diff': mean_diff,
            't_test': {'statistic': t_stat, 'p_value': t_pval},
            'wilcoxon': {'statistic': w_stat, 'p_value': w_pval},
            'significant_t': t_pval < alpha,
            'significant_w': w_pval < alpha
        }

        print(f"\n{model1} vs {model2}:")
        print(f"   Mean difference: {mean_diff:.4f}")
        print(f"   T-test p-value: {t_pval:.4f} {'(significant)' if t_pval < alpha else ''}")
        print(f"   Wilcoxon p-value: {w_pval:.4f} {'(significant)' if w_pval < alpha else ''}")

    return test_results


def friedman_test(results_dict, alpha=0.05):
    """
    Perform the Friedman test to compare more than two models across multiple datasets/folds.

    This is the non-parametric equivalent of repeated-measures ANOVA.

    Parameters:
    -----------
    results_dict : dict
        Dict containing each model name and its 'cv_scores'
    alpha : float
        Significance level

    Returns:
    --------
    (statistic, p_value, is_significant)
    """

    print("\nFRIEDMAN TEST (Multiple Model Comparison)")
    print("-" * 40)

    # Collect all models' cv scores into a list
    model_scores = [results_dict[m]['cv_scores'] for m in results_dict.keys()]

    # Apply Friedman test
    stat, p_value = friedmanchisquare(*model_scores)
    is_significant = p_value < alpha

    print(f"   Friedman statistic: {stat:.4f}")
    print(f"   P-value: {p_value:.4f}")
    print(f"   Significant difference: {'Yes' if is_significant else 'No'} (α={alpha})")

    return stat, p_value, is_significant


def effect_size_analysis(results_dict):
    """
    Compute Cohen's d effect size for all pairwise model comparisons.
    This tells how *big* the performance difference is, not just whether it's significant.

    Guidelines for interpretation (absolute value of d):
    - < 0.2  : negligible effect
    - < 0.5  : small effect
    - < 0.8  : medium effect
    - >= 0.8 : large effect

    Parameters:
    -----------
    results_dict : dict
        Dict containing each model's cross-validation scores.

    Returns:
    --------
    dict
        {
            "ModelA_vs_ModelB": {"cohens_d": value, "interpretation": "small/medium/..."},
            ...
        }
    """
    print("\nEFFECT SIZE ANALYSIS (Cohen's d)")
    print("-" * 40)

    model_names = list(results_dict.keys())
    effect_sizes = {}

    for model1, model2 in itertools.combinations(model_names, 2):
        scores1 = results_dict[model1]['cv_scores']
        scores2 = results_dict[model2]['cv_scores']

        pooled_std = np.sqrt((np.var(scores1) + np.var(scores2)) / 2)
        d = (np.mean(scores1) - np.mean(scores2)) / pooled_std

        if abs(d) < 0.2:
            interpretation = "negligible"
        elif abs(d) < 0.5:
            interpretation = "small"
        elif abs(d) < 0.8:
            interpretation = "medium"
        else:
            interpretation = "large"

        effect_sizes[f"{model1}_vs_{model2}"] = {
            'cohens_d': d,
            'interpretation': interpretation
        }

        print(f"   {model1} vs {model2}: d = {d:.3f} ({interpretation})")

    return effect_sizes


In [ ]:
# =============================================================================
# BAYESIAN HYPERPARAMETER TUNING (Optimized & Cleaned)
# =============================================================================

def optimized_bayesian_hyperparameter_tuning(X_train, y_train, X_val, y_val):
    """
    Perform efficient Bayesian hyperparameter tuning for multiple models
    (Logistic Regression, LightGBM, XGBoost) using reduced search spaces 
    and limited iterations to control compute time.

    Parameters:
    -----------
    X_train, y_train : Training data
    X_val, y_val     : Validation data (kept for future extensions, not used here)

    Returns:
    --------
    results : dict
        Dictionary containing best estimator, best score, and params for each model.
    """

    print("OPTIMIZED BAYESIAN HYPERPARAMETER TUNING")
    print("=" * 60)

    # Define models and reduced parameter spaces
    models_and_params = {
        'Logistic Regression': {
            'model': LogisticRegression(random_state=42, max_iter=1000),
            'params': {
                'C': Real(1e-3, 1e2, prior='log-uniform'),
                'penalty': Categorical(['l1', 'l2']),
                'solver': Categorical(['liblinear', 'saga']),
                'class_weight': Categorical([None, 'balanced'])
            }
        },
        'LightGBM': {
            'model': lgb.LGBMClassifier(random_state=42, verbose=-1),
            'params': {
                'n_estimators': Integer(50, 200),
                'learning_rate': Real(0.05, 0.3),
                'max_depth': Integer(3, 8),
                'num_leaves': Integer(15, 60),
                'min_child_samples': Integer(10, 40),
                'subsample': Real(0.7, 1.0),
                'colsample_bytree': Real(0.7, 1.0),
                'reg_alpha': Real(0.01, 10),
                'reg_lambda': Real(0.01, 10)
            }
        },
        'XGBoost': {
            'model': xgb.XGBClassifier(random_state=42, eval_metric='logloss'),
            'params': {
                'n_estimators': Integer(50, 200),
                'learning_rate': Real(0.05, 0.3),
                'max_depth': Integer(3, 8),
                'min_child_weight': Integer(1, 8),
                'subsample': Real(0.7, 1.0),
                'colsample_bytree': Real(0.7, 1.0),
                'reg_alpha': Real(0.01, 10),
                'reg_lambda': Real(0.01, 10),
                'gamma': Real(0, 2)
            }
        }
    }

    results = {}

    # Perform Bayesian Optimization
    for name, config in models_and_params.items():
        print(f"\nOptimizing {name}...")

        search = BayesSearchCV(
            estimator=config['model'],
            search_spaces=config['params'],
            n_iter=20,              # Reduced for faster tuning
            cv=3,                   # 3-fold cross validation
            scoring='f1_macro',
            random_state=42,
            n_jobs=2
        )

        search.fit(X_train, y_train)

        # Store model results
        results[name] = {
            'best_params': search.best_params_,
            'best_score': search.best_score_,
            'cv_scores': search.cv_results_['mean_test_score'],
            'best_estimator': search.best_estimator_
        }

        print(f"   Best CV F1-Macro: {search.best_score_:.4f}")
        print("   Best Parameters:")
        for param, value in search.best_params_.items():
            print(f"      {param}: {value}")

    return results


🖥️ PERFORMANCE ESTIMATE
   Total estimated runtime: ~30 minutes
   RAM usage: ~3-4 GB
   CPU usage: 50% (2 cores)
   Total evaluations: 60 (20 per model)
🎯 OPTIMIZED BAYESIAN HYPERPARAMETER TUNING

🔄 Optimizing Logistic Regression with Bayesian Search...
   🎯 Running 20 Bayesian iterations with 3-fold CV
   ⏱️ Estimated time: 3 minutes
   ✅ Best CV Score: 0.8855
   🎯 Best Parameters:
      C: 19.10178715209079
      class_weight: None
      penalty: l1
      solver: liblinear

🔄 Optimizing LightGBM with Bayesian Search...
   🎯 Running 20 Bayesian iterations with 3-fold CV
   ⏱️ Estimated time: 12 minutes
   ✅ Best CV Score: 0.8163
   🎯 Best Parameters:
      colsample_bytree: 0.9103661677432962
      learning_rate: 0.29888282209418915
      max_depth: 4
      min_child_samples: 21
      n_estimators: 101
      num_leaves: 60
      reg_alpha: 0.01
      reg_lambda: 10.0
      subsample: 0.8195395088209111

🔄 Optimizing XGBoost with Bayesian Search...
   🎯 Running 20 Bayesian iterations 

In [ ]:
def save_tuned_models_and_params(results, save_dir=r'C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\data\tuned_models'):
    """
    Saves all tuned models, their best parameters, and tuning summaries to disk.

    Why this is useful:
    -------------------
    After hyperparameter tuning, you usually want to:
    - Save the best estimator (trained model).
    - Save the best hyperparameters.
    - Save the summary of all results for later loading, comparison or re-evaluation.

    This function ensures reproducibility — models can be reloaded later without re-tuning.

    Parameters:
    -----------
    results : dict
        Output from hyperparameter tuning step.
        Should have structure:
        {
            "ModelName": {
                'best_estimator': fitted_model_object,
                'best_params': {param_dict},
                'best_score': float,
                ... (optional other keys)
            },
            ...
        }

    save_dir : str
        Directory where models and results will be stored.

    Returns:
    --------
    saved_models : dict
        A dictionary summarizing saved file paths and best scores for each model.
    """
    import os

    # Create directory if it doesn't already exist
    os.makedirs(save_dir, exist_ok=True)

    print("Saving tuned models and parameters")
    print("=" * 50)

    saved_models = {}

    for model_name, model_results in results.items():
        print(f"\nSaving {model_name}...")

        # Create filesystem-friendly filename from model name
        clean_name = model_name.replace(' ', '_').lower()

        # Define storage paths
        model_path = os.path.join(save_dir, f'{clean_name}_best_model.pkl')
        params_path = os.path.join(save_dir, f'{clean_name}_best_params.pkl')
        results_path = os.path.join(save_dir, f'{clean_name}_results.pkl')

        # Save model, parameters, and full results
        joblib.dump(model_results['best_estimator'], model_path)
        joblib.dump(model_results['best_params'], params_path)
        joblib.dump(model_results, results_path)

        # Add to summary dictionary
        saved_models[model_name] = {
            'model_path': model_path,
            'params_path': params_path,
            'results_path': results_path,
            'best_score': model_results['best_score'],
            'best_params': model_results['best_params']
        }

        print(f"   Model saved to:      {model_path}")
        print(f"   Parameters saved to: {params_path}")
        print(f"   Results saved to:    {results_path}")
        print(f"   Best CV F1-Macro:    {model_results['best_score']:.4f}")

    # Save overall summary for easy loading later
    summary_path = os.path.join(save_dir, 'tuning_summary.pkl')
    joblib.dump(saved_models, summary_path)

    print("\nSummary of tuned models:")
    print("-" * 30)

    # Rank models by best score
    sorted_models = sorted(saved_models.items(), key=lambda x: x[1]['best_score'], reverse=True)
    for i, (name, info) in enumerate(sorted_models, 1):
        print(f"   {i}. {name}: {info['best_score']:.4f}")

    print(f"\nAll models and parameters saved to: {save_dir}")
    print(f"Summary saved to: {summary_path}")

    return saved_models


In [14]:
saved_models_info = save_tuned_models_and_params(results)



💾 SAVING TUNED MODELS AND PARAMETERS

📁 Saving Logistic Regression...
   ✅ Model saved: C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\data\tuned_models\logistic_regression_best_model.pkl
   ✅ Parameters saved: C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\data\tuned_models\logistic_regression_best_params.pkl
   ✅ Results saved: C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\data\tuned_models\logistic_regression_results.pkl
   📊 Best CV F1-Macro: 0.8855

📁 Saving LightGBM...
   ✅ Model saved: C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\data\tuned_models\lightgbm_best_model.pkl
   ✅ Parameters saved: C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\data\tuned_models\lightgbm_best_params.pkl
   ✅ Results saved: C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\data\tuned_models\lightgbm_results.pkl
   📊 Best CV F1-Macro: 0.8163

📁 Saving XGBoost...
   ✅ Model saved: C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\data\tuned_models\xgboost_be